In [1]:
import torch
import torch.nn as nn

In [2]:
from deps.other_components import FeedForward_SwiGLU

In [3]:
class RMSNorm_Qwen(nn.Module):
    def __init__(self, emb_dim, eps=1e-6, bias=False):
        super().__init__()

        self.eps = eps
        # Llama had only 1 parameter
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim)) if bias else None
    
    def forward(self, x):
        input_dtype = x.dtype
        x = x.to(torch.float32)  # for Qwen
        
        # RMSNorm
        variance = x.pow(2).mean(dim=-1, keepdim=True)
        norm_x = x * torch.rsqrt(variance + self.eps)

        # new
        norm_x = norm_x * self.scale
        if self.shift is not None:
            norm_x = norm_x + self.shift
        
        return norm_x.to(input_dtype)